<a href="https://colab.research.google.com/github/bagustris/ravdess_song/blob/main/ravdess_song_sd_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAVDESS Song Emotion Recognition with LSTM 
Acoustic features are loaded directly from data directory

In [1]:
import glob
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [2]:
#!git clone https://github.com/bagustris/ravdess_song.git

In [3]:
#cd ravdess_song/

In [4]:
# assign hsf to X, lab to y; reshape X for LSTM
X = np.load('data/x.npy', allow_pickle=True)
y = np.load('data/y.npy', allow_pickle=True)

In [5]:
# reshape x untuk lstm
X = X.reshape((X.shape[0], 1, X.shape[1]))

In [11]:
X.shape

(1012, 1, 386)

In [12]:
y.shape

(1012,)

In [6]:
# split into train and test
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [13]:
x_train.shape

(910, 1, 386)

In [14]:
y_train.shape

(910,)

In [15]:
x_test.shape

(102, 1, 386)

In [16]:
y_test.shape

(102,)

In [7]:
# functio to build LSTM model
def model_lstm():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.BatchNormalization(axis=-1,
              input_shape=(x_train.shape[1], x_train.shape[2])))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.LSTM(256, return_sequences=True))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(6, activation='softmax'))

    # compile model: set loss, optimizer, metric
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  optimizer=tf.keras.optimizers.Adam(),
                  metrics=['accuracy'])
    return model

In [8]:
# create the model
model = model_lstm()
print(model.summary())
# hasil print model. shape dense hanya 6 sesuai dengan outputnya yaitu 6 emosi dalam song

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 1, 386)            1544      
_________________________________________________________________
lstm (LSTM)                  (None, 1, 256)            658432    
_________________________________________________________________
lstm_1 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
lstm_2 (LSTM)                (None, 1, 256)            525312    
_________________________________________________________________
flatten (Flatten)            (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 6)                 1

In [9]:
# train the LSTM model
hist = model.fit(x_train, y_train, epochs=100, shuffle=True, batch_size=16)

Epoch 1/100
57/57 [==============================] - 20s 106ms/step - loss: 1.4183 - accuracy: 0.4308
Epoch 2/100
57/57 [==============================] - 6s 108ms/step - loss: 0.7113 - accuracy: 0.7275
Epoch 3/100
57/57 [==============================] - 5s 93ms/step - loss: 0.4042 - accuracy: 0.8516
Epoch 4/100
57/57 [==============================] - 6s 109ms/step - loss: 0.3740 - accuracy: 0.8626
Epoch 5/100
57/57 [==============================] - 5s 87ms/step - loss: 0.2786 - accuracy: 0.8890
Epoch 6/100
57/57 [==============================] - 4s 69ms/step - loss: 0.2342 - accuracy: 0.9066
Epoch 7/100
57/57 [==============================] - 3s 55ms/step - loss: 0.2744 - accuracy: 0.9110
Epoch 8/100
57/57 [==============================] - 4s 74ms/step - loss: 0.1569 - accuracy: 0.9385
Epoch 9/100
57/57 [==============================] - 3s 50ms/step - loss: 0.1887 - accuracy: 0.9275
Epoch 10/100
57/57 [==============================] - 4s 69ms/step - loss: 0.1608 - accuracy: 0.

In [10]:
evaluate = model.evaluate(x_test, y_test, batch_size=16)
print("Loss: ", evaluate[0], "--> Accuracy: ", evaluate[1])

7/7 [==============================] - 5s 24ms/step - loss: 0.1937 - accuracy: 0.9608
Loss:  0.1936584711074829 --> Accuracy:  0.9607843160629272
